In [30]:
from gssutils import *

# Scrape details of all the datasets
scraper = Scraper('https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Pages/OTSReports.aspx')
scraper

## HM Revenue & Customs uktradeinfo - OTS Reports

This is a catalog of datasets; choose one from the following:

* Midlands Regional Trade Statistics data 2014-2016
* 2015 UK VAT Registered Importer and Exporter Population
* Overseas Trade Statistics broken down by English Growth Hub areas - EU and non-EU Exports 2015
* UK Trade in Goods by Business Characteristics - Experimental Statistics
* 2010 Quinquennial Review of the UK Ancillary Cost Survey (ACS) full report
* Quality Report
* UK Statistics article
* 2009 Intrastat Triennial Final Report
* EDICOM Technical Implementation Report
* 2005 Intrastat Triennial Review V2.0
* EDICOM Report
* EDICOM Report

In [31]:
# From that, narrow down to the specific dataset we want
scraper.select_dataset(title=lambda x: x.startswith('UK Trade in Goods by Business Characteristics'))
scraper

## UK Trade in Goods by Business Characteristics - Experimental Statistics

### Distributions

1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2015.pdf))
1. IDBR OTS tables 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2015.xls))
1. IDBR OTS Country data (expert users) 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2015.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2014.pdf))
1. IDBR OTS tables 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2014.xls))
1. IDBR OTS Country data (expert users) 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2014.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2013.pdf))
1. IDBR OTS tables 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2013.xls))
1. IDBR OTS Country data (expert users) 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_Country_data%20_expert_2013.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_2012.pdf))
1. IDBR OTS tables 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_tables_2009_to_2012.xls))
1. IDBR OTS Country data (expert users) 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_Country_2009_to_2012.xls))


In [32]:
# Select the distribution we need from that dataset
distributions_chosen = [dist for dist in scraper.distributions if dist.title.startswith('IDBR OTS tables')]

# TODO - how do we handle time in this?
# IDBR OTS tables 2009 - 2012

# sanity check, print distribution names 
distributions_chosen = [x for x in distributions_chosen if x.title != "IDBR OTS tables 2009 - 2012"]

# Sanity check - print the distributions being processed
[x.title for x in distributions_chosen]

['IDBR OTS tables 2015', 'IDBR OTS tables 2014', 'IDBR OTS tables 2013']

### params_per_tab
The tabs are almost identically strucuturd save a few reference points in different places, so we'll create a dictionary of parameters per-tab as it'll massively simplify the code.

_note - I've used the "Box" library to add convenient dot notation_

In [33]:
from box import Box

params_per_tab = {
    "Age Group": Box({
        "expected_obs": 42,
        "flow_cols": ["C", "E"],
        "metric_1": "Age of Business",
        "metric_2": None,
        "mixed_measures": {
            "Total value of UK trade by Age of Business": {'unit': 'Employees', "measure_type": "Employees Count"}, 
            "Business count by Age of Business": {'unit': 'Businesses', "measure_type": "Businesses Count"},
            "Employee count for Businesses by Age of Business": {'unit': 'Employees', "measure_type": "Employees Count"}
        }
    }),
    "Employee Size": Box({
        "expected_obs": 36,
        "flow_cols": ["C", "E"],
        "metric_1": "Employment",
        "metric_2": None,
        "mixed_measures": {
            "Total value of UK trade by Employee Size": {'unit': 'Employees', "measure_type": "Employees Count"}, 
            "Business count by Employee Size": {'unit': 'Businesses', "measure_type": "Businesses Count"},
            "Employee count for Businesses by Employee Size": {'unit': 'Employees', "measure_type": "Employees Count"}
        }
    }),
    "Industry Group": Box({
        "expected_obs": 66,
        "flow_cols": ["C", "E"],
        "metric_1": "HMRC Industry",
        "metric_2": None,
        "mixed_measures": {
            "Total value of UK trade by Industry Group": {'unit': 'Employees', "measure_type": "Employees Count"}, 
            "Business count by Industry Group": {'unit': 'Businesses', "measure_type": "Businesses Count"},
            "Employee count for Businesses by Industry Group": {'unit': 'Employees', "measure_type": "Employees Count"}
        }
    }),
    "Industry_Age": Box({
        "expected_obs": 66,
        "flow_cols": ["D", "H"],
        "metric_1": "Age of Business",
        "metric_2": "HMRC Industry",
        "measures": {
            "unit": "todo",
            "measure_type": "also todo"
        }
    })
}

### processing

this is our main processing loop. post processing will be handled in a later cell.

In [38]:

ignore_these_tabs = ["Notes and Contents", "Metadata"]

tidy_sheets = []

# for each distribution
for distribution in distributions_chosen:

    # get the tabs
    tabs = [tab for tab in distribution.as_databaker() if tab.name not in ignore_these_tabs]
    
    TEMP = ["Age Group", "Employee Size", "Industry Group", "Industry_Age"]
    tabs = [x for x in tabs if x.name in TEMP]

    # then process each in turn
    for tab in tabs:

        # log out some useful things
        this_task = "Processing >> Distribution: '{}', Tab: '{}'".format(distribution.title, tab.name)
        params = params_per_tab[tab.name]
        print(this_task)
        print("Params", params)

        # get the year from the end of the distribution name
        year = distribution.title[-4:]
        assert int(year) > 2005 and int(year) < 2025, "failed to get year from end of: {}, in {}.".format(
            distribution.title, this_task) 

        # Dimension - flow
        # Note - we're taking this early as it's useful when selecting dimensions
        flow = tab.excel_ref(params.flow_cols[0]).filter("Exports") | tab.excel_ref(params.flow_cols[1]).filter("Imports")
        importExportVals = [cell.value for cell in flow]
        assert  'Imports' in importExportVals and 'Exports' in importExportVals, \
            "the flow cells should contain the values 'Imports' and 'Exports'. in:".format(this_task)

        # Observations
        observations = tab.excel_ref('C') | tab.excel_ref('E')
        observations = observations.is_not_bold().regex("^\d*\.?\d*$")
        observations = observations.is_not_blank() - flow.shift(DOWN) # get rid of the years
        observations = observations - tab.excel_ref('A').filter(contains_string("Total")).expand(RIGHT)
        #assert len(observations) == params.expected_obs, "Failing to get expected number of " \
        #    "observations, tab: {}, expected {}. got {}.".format(tab.name, params.expected_obs, len(observations))
            
        # Dimension - the dynamic metrics, changes per tab
        if params.metric_2 == None:
            metric_1_cells = tab.excel_ref('A').expand(DOWN)
        else:
            metric_1_cells = tab.excel_ref('B').expand(DOWN)
            metric_2_cells = tab.excel_ref('A').is_not_blank()            

        dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDimConst('Year', year),          
            HDim(metric_1_cells, params.metric_1, DIRECTLY, LEFT),
            HDim(flow, 'Flow', DIRECTLY, ABOVE)
            ]
        
        # For multi-measure tabs, get the text indicating which measures to use
        if "mixed_measures" in params.keys():
            cell_text = list(params.measures.keys())
            measures = tab.filter(cell_text[0]).assert_one() | \
                        tab.filter(cell_text[1]).assert_one() | \
                        tab.filter(cell_text[2]).assert_one()
            dimensions.append(HDim(measures, "Measure Type", CLOSEST, ABOVE))
            dimensions.append(HDim(measures, "Unit", CLOSEST, ABOVE))
        else:
            dimensions.append(HDimConst("Measure Type", params.measures.measure_type))
            dimensions.append(HDimConst("Unit", params.measures.measure_type))
        
        # for the tabs with two dynamic metrics (the ones with underscores in the name)
        # we also need to use the 2nd metric
        if params.metric_2 != None:
            dimensions.append(HDim(metric_2_cells, params.metric_2, CLOSEST, ABOVE))

        cs = ConversionSegment(observations, dimensions)

        # We're going to have a LOT of previews, so outputting them to files
        savepreviewhtml(cs, fname="{}_{}.html".format(distribution.title, tab.name), verbose=False)

        # Add the measures and get rid of our temp column
        tidy_sheet = cs.topandas()
        tidy_sheet["Unit"] = tidy_sheet["Unite"].map(lambda x: params.mixed_measures[x].unit)
        tidy_sheet["Measure Type"] = tidy_sheet["Measure Type"].map(lambda x: params.mixed_measures[x].measure_type)

        tidy_sheets.append(tidy_sheet)


Processing >> Distribution: 'IDBR OTS tables 2015', Tab: 'Industry Group'
Params {'expected_obs': 66, 'flow_cols': ['C', 'E'], 'metric_1': 'HMRC Industry', 'metric_2': None, 'measures': {'Total value of UK trade by Industry Group': {'unit': 'Employees', 'measure_type': 'Employees Count'}, 'Business count by Industry Group': {'unit': 'Businesses', 'measure_type': 'Businesses Count'}, 'Employee count for Businesses by Industry Group': {'unit': 'Employees', 'measure_type': 'Employees Count'}}}


BoxKeyError: "'Box' object has no attribute 'measure_type'"

In [15]:
# post processing

import pandas as pd
combined_tidy_sheet = pd.concat(tidy_sheets, sort=False)

# for HMRC Industry we just want "group x" so "Group 9: FooBar!" becomes "group-9"
combined_tidy_sheet["HMRC Industry"] = combined_tidy_sheet["HMRC Industry"].map(lambda x: str(x)[:7].lower().replace(" ", "-"))



array(['group-1', 'group-2', 'group-3', 'group-4', 'group-5', 'group-6',
       'group-7', 'group-8', 'group-9', 'unknown', 'nan'], dtype=object)

In [7]:

for col in combined_tidy_sheet.columns.values:
    if col != "OBS":
        print(col, " "*(20-len(col)), list(combined_tidy_sheet[col].unique()))
    
# Output a few lines as a preview
combined_tidy_sheet

DATAMARKER            [nan, '.']
Geography             ['K02000001']
Year                  ['2015', '2014', '2013']
Measure Type          ['Employees Count', 'Businesses Count']
HMRC Industry         ['Group 1 Agriculture and Food', 'Group 2 Mining, Petroleum products and Waste', 'Group 3 Chemicals', 'Group 4 Pharmaceuticals', 'Group 5 Electronic and Electrical equipment', 'Group 6 Machinery and equipment n.e.s', 'Group 7 Motor vehicles, transport equipment (excluding aerospace)', 'Group 8 Aerospace and related machinery', 'Group 9 Other manufacturing', 'Group 10 Services', 'Unknown', nan]
Flow                  ['Exports', 'Imports']
Unit                  ['Employees', 'Businesses']
Age of Business       [nan, '0 to 1', '2 to 3', '4 to 5', '6 to 9 ', '10 to 20', '20 +', 'Unknown', 'Unknown8']
Employment            [nan, '0.0', '1 to 9', '10 to 49', '50 to 249', '250 +', 'Unknown']


,OBS,DATAMARKER,Geography,Year,Measure Type,HMRC Industry,Flow,Unit,Age of Business,Employment
0,15178,NaN,K02000001,2015,Employees Count,Group 1 Agriculture and Food,Exports,Employees,NaN,NaN
1,18388,NaN,K02000001,2015,Employees Count,Group 1 Agriculture and Food,Imports,Employees,NaN,NaN
2,17836,NaN,K02000001,2015,Employees Count,"Group 2 Mining, Petroleum products and Waste",Exports,Employees,NaN,NaN
3,23226,NaN,K02000001,2015,Employees Count,"Group 2 Mining, Petroleum products and Waste",Imports,Employees,NaN,NaN
4,13564,NaN,K02000001,2015,Employees Count,Group 3 Chemicals,Exports,Employees,NaN,NaN
5,9467,NaN,K02000001,2015,Employees Count,Group 3 Chemicals,Imports,Employees,NaN,NaN
6,10025,NaN,K02000001,2015,Employees Count,Group 4 Pharmaceuticals,Exports,Employees,NaN,NaN
7,3154,NaN,K02000001,2015,Employees Count,Group 4 Pharmaceuticals,Imports,Employees,NaN,NaN
8,17912,NaN,K02000001,2015,Employees Count,Group 5 Electronic and Electrical equipment,Exports,Employees,NaN,NaN
9,13696,NaN,K02000001,2015,Employees Count,Group 5 Electronic and Electrical equipment,Imports,Employees,NaN,NaN
